In [28]:
import nltk
from collections import defaultdict
import pandas as pd

In [29]:
from nltk.corpus import stopwords

In [30]:
from nltk.tokenize import TweetTokenizer

In [31]:
import fighting_words

In [32]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shengruilyu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [42]:
stop_words = set(stopwords.words('english'))
stop_words.add('removed')

In [35]:
df = pd.read_csv("../after_regex_large.csv", index_col = 0, low_memory = False)

In [36]:
tknzr = TweetTokenizer()

In [37]:
word_frequency_by_subreddit= {}

In [38]:
# topic_postags = set(['NN', 'NNS', 'NNP', 'NNPS'])

In [39]:
tokens_by_subreddit = {}

In [40]:
posts_by_subreddit = {}

In [41]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    if sub_reddit in posts_by_subreddit:
        posts_by_subreddit[sub_reddit].append(row['body'])
    else:
        posts_by_subreddit[sub_reddit] = [row['body']]
    tokens = tknzr.tokenize(row['body'])
    if sub_reddit in tokens_by_subreddit:
        tokens_by_subreddit[sub_reddit].append(tokens)
    else:
        tokens_by_subreddit[sub_reddit] = [tokens]
    
    if sub_reddit in word_frequency_by_subreddit:
        word_frequency = word_frequency_by_subreddit[sub_reddit]
    else:
        word_frequency = defaultdict(int)
        word_frequency_by_subreddit[sub_reddit] = word_frequency
    for token in tokens:
        if (token not in stop_words) and (token.isalnum()):
            word_frequency[token]+=1
    

In [43]:
subreddits = list(posts_by_subreddit.keys())

In [44]:
subreddits

['changemyview',
 'YouShouldKnow',
 'AskMen',
 'Futurology',
 'Art',
 'IAmA',
 'science',
 'Cooking']

In [55]:
topic_per_by_subreddit = {}
results_by_subreddit = {}

In [56]:
for current_subreddit in subreddits:
    l1 = posts_by_subreddit[current_subreddit]
    l2 = []
    for sr in subreddits:
        if sr == current_subreddit:
            print (sr)
            continue
        l2 += posts_by_subreddit[sr]
    results = fighting_words.bayes_compare_language(l1, l2)
    sort_results = sorted(results, key = lambda x : x[1], reverse = True)
    results_by_subreddit[current_subreddit] = sort_results
    

changemyview
Vocab size is 15000
Comparing language...
YouShouldKnow
Vocab size is 15000
Comparing language...
AskMen
Vocab size is 15000
Comparing language...
Futurology
Vocab size is 15000
Comparing language...
Art
Vocab size is 15000
Comparing language...
IAmA
Vocab size is 15000
Comparing language...
science
Vocab size is 15000
Comparing language...
Cooking
Vocab size is 15000
Comparing language...


In [65]:
duplicate = set([])
all_topics = set([])
for sr in results_by_subreddit.keys():
    for topic in topic_per_by_subreddit[sr]:
        if topic in all_topics:
            duplicate.add(topic)
        all_topics.add(topic)

In [83]:
duplicate

{'animals',
 'cats',
 'energy',
 'got',
 'guys',
 'like',
 'looks',
 'love',
 'meat',
 'nuclear',
 'thank',
 'thanks',
 'virus',
 'women'}

In [84]:
# remove from duplicate
duplicate.remove('animals')
duplicate.remove('cats')
duplicate.remove('energy')
duplicate.remove('meat')
duplicate.remove('nuclear')
duplicate.remove('virus')
duplicate.remove('women')

In [85]:
duplicate

{'got', 'guys', 'like', 'looks', 'love', 'thank', 'thanks'}

In [86]:
remove_duplicate_topic = True

In [99]:
topk = 100

In [100]:
for current_subreddit in subreddits:
    topics = []
    sort_results = results_by_subreddit[current_subreddit]
    for word, z in sort_results:
        if len(topics) == topk:
            break
        if word in stop_words:
            continue
        if remove_duplicate_topic and word in duplicate:
            continue
        topics.append(word)
    topic_per_by_subreddit[current_subreddit] = topics

In [101]:
topic_per_by_subreddit

{'changemyview': ['view',
  'trump',
  'biden',
  'vote',
  'comments',
  'rule',
  'op',
  'argument',
  'must',
  'explained',
  'within',
  'comment',
  'delta',
  'breaking',
  'page',
  'message',
  'posted',
  'party',
  'review',
  'moderators',
  'appeal',
  'gender',
  'notice',
  'wiki',
  'standards',
  'ban',
  'gun',
  'note',
  'believe',
  'sexual',
  'user',
  'link',
  'arguments',
  'process',
  'voting',
  'change',
  'rights',
  'point',
  'information',
  'person',
  'nt',
  'women',
  'system',
  'favor',
  'argue',
  'lead',
  'trans',
  'agree',
  'clicking',
  'evidence',
  'rape',
  'multiple',
  'moral',
  'would',
  'crime',
  'conversation',
  'bernie',
  'election',
  'democrats',
  'men',
  'rude',
  'stated',
  'example',
  'court',
  'appeals',
  'aspect',
  'voters',
  'definition',
  'consent',
  'minor',
  'people',
  'police',
  'gay',
  'candidate',
  'president',
  'willing',
  'falls',
  'moderation',
  'assault',
  'victim',
  'progressive',
  '

In [102]:
for index, row in df.iterrows():
    sub_reddit = row['subreddit']
    tokens = tknzr.tokenize(row['body'])
    new_tokens = []
    for token in tokens:
        if token in topic_per_by_subreddit[sub_reddit]:
            new_tokens.append("TOPIC")
        else:
            new_tokens.append(token)
    row['body'] = " ".join(new_tokens)

In [103]:
df.head()

,body,subreddit,author
0,I thought this was already the case ? Had it b...,changemyview,dublea
1,The original post had the moratorum lasting un...,changemyview,Ansuz07
2,"Technically , the H1N1 pandemic of 2009 was on...",changemyview,scott60561
3,The WHO declared an end of the H1N1 pandemic (...,changemyview,SaxonySam
4,Thank you for providing TOPIC with sources So ...,changemyview,epmuscle


In [104]:
if remove_duplicate_topic:
    filename = 'after_non_duplicate_top' + str(topk) + '_removal_large.csv'
else: 
    filename = 'after_top' + str(topk) + '_removal_large.csv'
df.to_csv(filename)